<a href="https://colab.research.google.com/github/nitesis/CAS_AICP_M6_Exercises/blob/main/Session_2_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Networks Classification task
We continue to work on the [IMBD Movie genre classification task](https://www.google.com/url?q=https%3A%2F%2Fhuggingface.co%2Fdatasets%2Fadrienheymans%2Fimdb-movie-genres).

In this tutorial,
1. At first we will use GLoVe to embedd the dataset and make a simple neural network for the classification task.
2. Then we will create an an LSTM model with GLoVe
3. Then we compare the performance of the models with previous models.

In [ ]:
!pip install torch torchvision torchaudio
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from datasets import load_dataset, DatasetDict
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time

In [ ]:
import urllib.request
import zipfile
import os

# Download GloVe embeddings
url = "https://nlp.stanford.edu/data/glove.6B.zip"
output_path = "glove.6B.zip"

if not os.path.exists(output_path):
    print("Downloading GloVe embeddings...")
    urllib.request.urlretrieve(url, output_path)

# Unzip the file
with zipfile.ZipFile(output_path, "r") as zip_ref:
    zip_ref.extractall("./glove/")
    print("GloVe embeddings extracted!")

GloVe embeddings extracted!


In [ ]:

# 1. Load GloVe Embeddings


# Path to GloVe file
glove_path = "./glove/glove.6B.50d.txt"

# Load GloVe embeddings into a dictionary
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]  # The word
            vector = np.asarray(values[1:], dtype='float32')  # The embedding vector
            embeddings[word] = vector
    return embeddings

# Load embeddings
glove_embeddings = load_glove_embeddings(glove_path)




In [ ]:
data = load_dataset("adrienheymans/imdb-movie-genres")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

(…)-00000-of-00001-b7b538a3d562331b.parquet:   0%|          | 0.00/23.2M [00:00<?, ?B/s]

(…)-00000-of-00001-8b6fe98b0dbe46a8.parquet:   0%|          | 0.00/23.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/54214 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/54200 [00:00<?, ? examples/s]

In [ ]:
genre_counts = data["train"].to_pandas()["genre"].value_counts()
genre_counts

,count
genre,
drama,13613
documentary,13096
comedy,7447
short,5073
horror,2204
thriller,1591
action,1315
western,1032
reality-tv,884


In [ ]:

# Get the top 4 most frequent genres
top_4_genres = genre_counts.head(4).index.tolist()

# Filter the training and test datasets
filtered_train_data = data["train"].filter(lambda example: example['genre'] in top_4_genres)
filtered_test_data = data["test"].filter(lambda example: example['genre'] in top_4_genres)

print(f"Filtered training dataset size: {len(filtered_train_data)}")
print(f"Filtered test dataset size: {len(filtered_test_data)}")

data = DatasetDict({
    'train': filtered_train_data,
    'test': filtered_test_data
})
data

Filter:   0%|          | 0/54214 [00:00<?, ? examples/s]

Filter:   0%|          | 0/54200 [00:00<?, ? examples/s]

Filtered training dataset size: 39229
Filtered test dataset size: 39226


DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'genre', 'label'],
        num_rows: 39229
    })
    test: Dataset({
        features: ['title', 'text', 'genre', 'label'],
        num_rows: 39226
    })
})

In [ ]:
target_classes = data["train"].to_pandas()["genre"].unique()
num_classes = len(target_classes)
num_classes

4

In [ ]:
genre_mapping = {genre: i for i, genre in enumerate(target_classes)} #maps labels to numeric IDs
genre_mapping

{'drama': 0, 'documentary': 1, 'comedy': 2, 'short': 3}

In [ ]:

# 2. Dataset Class
class TextDataset(Dataset):
    def __init__(self, texts, labels, embeddings, genre_mapping = genre_mapping):
        self.texts = texts
        self.labels = labels
        self.embeddings = embeddings
        self.genre_mapping = genre_mapping

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokens = self.texts[idx].split()
        # Average the embeddings for the sentence
        vectorized = [self.embeddings.get(word, np.zeros(50)) for word in tokens]
        vectorized = np.mean(vectorized, axis=0) if vectorized else np.zeros(50)
        # Convert label to numerical using the mapping
        label = self.genre_mapping[self.labels[idx]]

        return torch.tensor(vectorized, dtype=torch.float32), torch.tensor(label, dtype=torch.long)



In [ ]:

# 3. Define the Neural Network
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x


In [ ]:

# 4. Prepare Data
# Extract texts and labels

train_dataset = TextDataset(texts = data["train"]['text'], labels =  data["train"]['genre'], embeddings = glove_embeddings)
test_dataset = TextDataset(texts = data["test"]['text'], labels =  data["test"]['genre'], embeddings = glove_embeddings)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)


In [ ]:

# 5. Initialize Model, Loss, and Optimizer
embedding_dim = 50
input_dim = embedding_dim
hidden_dim = 128
output_dim = num_classes  # Number of classes
model = SimpleNN(input_dim, hidden_dim, output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:

# 6. Training Loop

epochs = 10
start = time.time()
for epoch in tqdm(range(1, epochs)):
    for inputs, labels in train_dataloader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")
print(f" training time in seconds {time.time() - start}")

 11%|█         | 1/9 [00:11<01:28, 11.02s/it]

Epoch 2/10, Loss: 0.8264787793159485


 22%|██▏       | 2/9 [00:20<01:09,  9.90s/it]

Epoch 3/10, Loss: 0.8892511129379272


 33%|███▎      | 3/9 [00:31<01:02, 10.43s/it]

...

Epoch 8/10, Loss: 1.0259712934494019


 89%|████████▉ | 8/9 [01:20<00:09,  9.84s/it]

Epoch 9/10, Loss: 1.0114402770996094


100%|██████████| 9/9 [01:32<00:00, 10.23s/it]

Epoch 10/10, Loss: 0.8227826952934265
 training time in seconds 92.04639148712158


In [ ]:
# Baseline Dummy Classifier
# Gibt Aussage über die Verteilung der Klassen und dient später als Richtwert für die Bewertung der Test Accuracy
# Mit [0] wird das Gengre mit der grössten Accuracy im Dummy Classifier aus der Liste genommen
data["test"].to_pandas()["genre"].value_counts()[0]/len(data["test"])

<ipython-input-23-3fa283d4c7c8>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data["test"].to_pandas()["genre"].value_counts()[0]/len(data["test"])


0.3470147351246622

In [ ]:
#7. Evaluate the performance
# Function to calculate accuracy
def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in dataloader:
            outputs = model(inputs)
            _, preds = torch.max(outputs, dim=1)  # Get the class with the highest probability
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
    # Calculate accuracy
    accuracy = accuracy_score(all_targets, all_preds)
    return accuracy

# Evaluate the model
accuracy = evaluate_model(model, test_dataloader)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 65.76%


# LSTM Network

In [ ]:
# 1. Define the LSTM Network
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1, bidirectional=False, dropout=0.5):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, bidirectional=bidirectional, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim * (2 if bidirectional else 1), output_dim)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # x shape: (batch_size, seq_len, input_dim)
        _, (hidden, _) = self.lstm(x)  # hidden shape: (num_layers * num_directions, batch_size, hidden_dim)
        if self.lstm.bidirectional:
            hidden = torch.cat((hidden[-2], hidden[-1]), dim=1)  # Combine final forward and backward hidden states
        else:
            hidden = hidden[-1]  # Use the last hidden state for unidirectional LSTM
        x = self.fc(hidden)  # Pass through fully connected layer
        x = self.softmax(x)
        return x


In [ ]:
#2. Define the dataset class
class SequentialTextDataset(Dataset):
    def __init__(self, texts, labels, embeddings, max_len=50):
        self.texts = texts
        self.labels = labels
        self.embeddings = embeddings
        self.max_len = max_len
        self.genre_mapping = genre_mapping

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokens = self.texts[idx].split()
        vectorized = [self.embeddings.get(word, np.zeros(50)) for word in tokens]

        # Pad or truncate to max_len
        if len(vectorized) < self.max_len:
            vectorized.extend([np.zeros(50)] * (self.max_len - len(vectorized)))
        else:
            vectorized = vectorized[:self.max_len]

        vectorized = np.array(vectorized)
        # Convert label to numerical using the mapping
        label = self.genre_mapping[self.labels[idx]]
        return torch.tensor(vectorized, dtype=torch.float32), torch.tensor(label, dtype=torch.long)


In [ ]:
#3. Create train and test datasets
train_dataset_lstm = SequentialTextDataset(data["train"]["text"], data["train"]["genre"], glove_embeddings)
train_loader_lstm = DataLoader(train_dataset_lstm, batch_size=32, shuffle=True)
test_dataset_lstm = SequentialTextDataset(data["test"]["text"], data["test"]["genre"], glove_embeddings)
test_loader_lstm = DataLoader(test_dataset_lstm, batch_size=32, shuffle=True)

In [ ]:

#4. Initialize Model, Loss, and Optimizer
input_dim = embedding_dim
hidden_dim = 128
output_dim = num_classes  # Number of classes
num_layers = 2
bidirectional = True
dropout = 0.5
max_len = 50  # Maximum sequence length

model_lstm = LSTMClassifier(input_dim, hidden_dim, output_dim, num_layers, bidirectional, dropout)
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_lstm = model_lstm.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_lstm.parameters(), lr=0.001)


In [ ]:
start = time.time()
# 5. Training Loop (Takes several minutes to run)
epochs = 10

for epoch in tqdm(range(1, epochs)):

    model_lstm.train()
    epoch_loss = 0.0
    for inputs, labels in train_loader_lstm:
        # Forward pass
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model_lstm(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_loader_lstm)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
print(f" training time in seconds {time.time() - start}")

 11%|█         | 1/9 [00:12<01:43, 12.89s/it]

Epoch 2/10, Loss: 1.0091


 22%|██▏       | 2/9 [00:21<01:11, 10.22s/it]

Epoch 3/10, Loss: 0.8988


 33%|███▎      | 3/9 [00:33<01:06, 11.11s/it]

...

Epoch 8/10, Loss: 0.7163


 89%|████████▉ | 8/9 [01:22<00:09,  9.86s/it]

Epoch 9/10, Loss: 0.6822


100%|██████████| 9/9 [01:34<00:00, 10.50s/it]

Epoch 10/10, Loss: 0.6397
 training time in seconds 94.53414249420166


In [ ]:
#6. Evaluate the performance
# Function to calculate accuracy
def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, dim=1)  # Get the class with the highest probability
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
    # Calculate accuracy
    accuracy = accuracy_score(all_targets, all_preds)
    return accuracy

# Evaluate the model
accuracy = evaluate_model(model_lstm, test_loader_lstm)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 67.28%


In [ ]:
text = "After his wife leaves him, a work-obsessed Manhattan advertising executive is forced to learn long-neglected parenting skills, but a heated custody battle over the couple's young son deepens the wounds left by the separation."
evaluate(text, model_lstm)

# To Do

Repeat the experiment for unfiltered data and compare with the results with the previous models from SVM with embeddings.

Which performed better? Discuss.
The unflitered is less better due to the fact that there are more classes and also more classes have maybe a less good accuracy


In [ ]:
# Solution ist to skip the following code in the first example:
# Get the top 4 most frequent genres
#top_4_genres = genre_counts.head(4).index.tolist()

# Filter the training and test datasets
#filtered_train_data = data["train"].filter(lambda example: example['genre'] in top_4_genres)
#filtered_test_data = data["test"].filter(lambda example: example['genre'] in top_4_genres)

#print(f"Filtered training dataset size: {len(filtered_train_data)}")
#print(f"Filtered test dataset size: {len(filtered_test_data)}")

#data = DatasetDict({
 #   'train': filtered_train_data,
 #   'test': filtered_test_data
#})
#data

In [ ]:
!pip install torch torchvision torchaudio
!pip install datasets

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from datasets import load_dataset, DatasetDict
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time

In [ ]:
import urllib.request
import zipfile
import os

# Download GloVe embeddings
url = "https://nlp.stanford.edu/data/glove.6B.zip"
output_path = "glove.6B.zip"

if not os.path.exists(output_path):
    print("Downloading GloVe embeddings...")
    urllib.request.urlretrieve(url, output_path)

# Unzip the file
with zipfile.ZipFile(output_path, "r") as zip_ref:
    zip_ref.extractall("./glove/")
    print("GloVe embeddings extracted!")

GloVe embeddings extracted!


In [ ]:
# 1. Load GloVe Embeddings


# Path to GloVe file
glove_path = "./glove/glove.6B.50d.txt"

# Load GloVe embeddings into a dictionary
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]  # The word
            vector = np.asarray(values[1:], dtype='float32')  # The embedding vector
            embeddings[word] = vector
    return embeddings

# Load embeddings
glove_embeddings = load_glove_embeddings(glove_path)

In [ ]:
data = load_dataset("adrienheymans/imdb-movie-genres")

In [ ]:
genre_counts = data["train"].to_pandas()["genre"].value_counts()
genre_counts

,count
genre,
drama,13613
documentary,13096
comedy,7447
short,5073
horror,2204
thriller,1591
action,1315
western,1032
reality-tv,884


In [ ]:
target_classes = data["train"].to_pandas()["genre"].unique()
num_classes = len(target_classes)
num_classes

27

In [ ]:
genre_mapping = {genre: i for i, genre in enumerate(target_classes)} #maps labels to numeric IDs
genre_mapping

{'drama': 0,
 'thriller': 1,
 'adult': 2,
 'documentary': 3,
 'comedy': 4,
 'crime': 5,
 'reality-tv': 6,
 'horror': 7,
 'sport': 8,
 'animation': 9,
 'action': 10,
 'fantasy': 11,
 'short': 12,
 'sci-fi': 13,
 'music': 14,
 'adventure': 15,
 'talk-show': 16,
 'western': 17,
 'family': 18,
 'mystery': 19,
 'history': 20,
 'news': 21,
 'biography': 22,
 'romance': 23,
 'game-show': 24,
 'musical': 25,
 'war': 26}

In [ ]:
# 2. Dataset Class
class TextDataset(Dataset):
    def __init__(self, texts, labels, embeddings, genre_mapping = genre_mapping):
        self.texts = texts
        self.labels = labels
        self.embeddings = embeddings
        self.genre_mapping = genre_mapping

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokens = self.texts[idx].split()
        # Average the embeddings for the sentence
        vectorized = [self.embeddings.get(word, np.zeros(50)) for word in tokens]
        vectorized = np.mean(vectorized, axis=0) if vectorized else np.zeros(50)
        # Convert label to numerical using the mapping
        label = self.genre_mapping[self.labels[idx]]

        return torch.tensor(vectorized, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

In [ ]:
# 3. Define the Neural Network
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

In [ ]:
# 4. Prepare Data
# Extract texts and labels

train_dataset = TextDataset(texts = data["train"]['text'], labels =  data["train"]['genre'], embeddings = glove_embeddings)
test_dataset = TextDataset(texts = data["test"]['text'], labels =  data["test"]['genre'], embeddings = glove_embeddings)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
# 5. Initialize Model, Loss, and Optimizer
embedding_dim = 50
input_dim = embedding_dim
hidden_dim = 128
output_dim = num_classes  # Number of classes
model = SimpleNN(input_dim, hidden_dim, output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 6. Training Loop

epochs = 10
start = time.time()
for epoch in tqdm(range(1, epochs)):
    for inputs, labels in train_dataloader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")
print(f" training time in seconds {time.time() - start}")

 11%|█         | 1/9 [00:20<02:45, 20.67s/it]

Epoch 2/10, Loss: 1.4960919618606567


 22%|██▏       | 2/9 [00:34<01:58, 16.91s/it]

Epoch 3/10, Loss: 1.941915512084961


 33%|███▎      | 3/9 [00:50<01:37, 16.27s/it]

...

Epoch 8/10, Loss: 0.821591854095459


 89%|████████▉ | 8/9 [02:06<00:16, 16.05s/it]

Epoch 9/10, Loss: 1.9167156219482422


100%|██████████| 9/9 [02:24<00:00, 16.02s/it]

Epoch 10/10, Loss: 1.7968721389770508
 training time in seconds 144.16330909729004


In [ ]:
# Baseline Dummy Classifier
# Gibt Aussage über die Verteilung der Klassen und dient später als Richtwert für die Bewertung der Test Accuracy
# Mit [0] wird das Gengre mit der grössten Accuracy im Dummy Classifier aus der Liste genommen
data["test"].to_pandas()["genre"].value_counts()[0]/len(data["test"])

<ipython-input-43-d52cdade1a76>:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data["test"].to_pandas()["genre"].value_counts()[0]/len(data["test"])


0.2511439114391144

In [ ]:
#7. Evaluate the performance
# Function to calculate accuracy
def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for inputs, targets in dataloader:
            outputs = model(inputs)
            _, preds = torch.max(outputs, dim=1)  # Get the class with the highest probability
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
    # Calculate accuracy
    accuracy = accuracy_score(all_targets, all_preds)
    return accuracy

# Evaluate the model
accuracy = evaluate_model(model, test_dataloader)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 51.30%
